In [1]:
def crop(img):
    roi=img[865:1602,1390:2364]
    return roi

In [103]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def pad_image(image, pad_size):
    return cv2.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_CONSTANT, value=(0, 0, 0))


In [3]:
def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

In [4]:
def random_rotate_image(image, max_angle=45):
    angle = np.random.randint(-max_angle, max_angle)
    return rotate_image(image, angle)


In [5]:
def scale_image(image, scale_factor):
    height, width = image.shape[:2]
    new_height, new_width = int(height * scale_factor), int(width * scale_factor)
    scaled_image = cv2.resize(image, (new_width, new_height))
    return scaled_image

In [6]:
def random_scale_image(image, min_scale=0.8, max_scale=1.2):
    scale_factor = np.random.uniform(min_scale, max_scale)
    return scale_image(image, scale_factor)


In [7]:
def random_translate_image(image, max_translate=50):
    height, width = image.shape[:2]
    tx = np.random.randint(-max_translate, max_translate)
    ty = np.random.randint(-max_translate, max_translate)
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, translation_matrix, (width, height))


In [8]:
def random_crop_image(image, crop_ratio=0.6):
    height, width = image.shape[:2]
    new_height, new_width = int(height * crop_ratio), int(width * crop_ratio)
    y = np.random.randint(0, height - new_height)
    x = np.random.randint(0, width - new_width)
    return image[y:y + new_height, x:x + new_width]
def randomcrop(img, gt_boxes, scale=0.5):
    '''
    ### Random Crop ###
    img: image
    gt_boxes: format [[obj x1 y1 x2 y2],...]
    scale: percentage of cropped area
    '''
    
    # Crop image
    height, width = int(img.shape[0]*scale), int(img.shape[1]*scale)
    x = random.randint(0, img.shape[1] - int(width))
    y = random.randint(0, img.shape[0] - int(height))
    cropped = img[y:y+height, x:x+width]
    resized = cv2.resize(cropped, (img.shape[1], img.shape[0]))
    
    # Modify annotation
    new_boxes=[]
    for box in gt_boxes:
        obj_name = box[0]
        x1 = int(box[1])
        y1 = int(box[2])
        x2 = int(box[3])
        y2 = int(box[4])
        x1, x2 = x1-x, x2-x
        y1, y2 = y1-y, y2-y
        x1, y1, x2, y2 = x1/scale, y1/scale, x2/scale, y2/scale
        if (x1<img.shape[1] and y1<img.shape[0]) and (x2>0 and y2>0):
            if x1<0: x1=0
            if y1<0: y1=0
            if x2>img.shape[1]: x2=img.shape[1]
            if y2>img.shape[0]: y2=img.shape[0]
            new_boxes.append([obj_name, x1, y1, x2, y2])
    return resized, new_boxes


In [9]:
def random_zoom_image(image, min_zoom=0.8, max_zoom=2.0):
    zoom_factor = np.random.uniform(min_zoom, max_zoom)
    return scale_image(image, zoom_factor)


In [10]:
def random_darken_brighten(image, brightness_range=(10, 30)):
    brightness = np.random.randint(brightness_range[0], brightness_range[1])
    return np.clip(image + brightness, 0, 255).astype(np.uint8)


In [11]:
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [112]:
import os
import cv2
import numpy as np
def augment_and_save(image, output_directory,a, num_images=3):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for i in range(num_images):
        rotated_image= random_rotate_image(image)
        grayscale_img= convert_to_grayscale(rotated_image)
        output_path = os.path.join(output_directory, f'3DIMG_15NOV2018_'+a+'_L1C_SGP_V01R00'+str(i)+'.jpg')
        cv2.imwrite(output_path, grayscale_img)

# Assuming 'original_img.jpg' is the input image
obj=['0300','0330','0400','0430','0500','0530','0600','0630','0700','0730','0800','0830','0900','0930']
for i in obj:

    input_image = cv2.imread('cropped/VIS/3DIMG_15NOV2018_'+i+'_L1C_SGP_V01R00.jpg')
    output_directory = 'aug_cropped/SCS/VIS'
    augment_and_save(input_image, output_directory,i)

# input_image = cv2.imread('cropped/MIR/3DIMG_15NOV2018_1800_L1C_SGP_V01R00.jpg')
# output_directory = 'aug_cropped/MIR'
# augment_and_save(input_image, output_directory)

# input_image = cv2.imread('cropped/WV/3DIMG_15NOV2018_1800_L1C_SGP_V01R00.jpg')
# output_directory = 'aug_cropped/WV'
# augment_and_save(input_image, output_directory)
